## Imports and working path

In [ ]:
# Imports
import os
import pandas as pd
from pathlib import Path

# Working path
base_dir = Path().resolve()
print(f'Working in {base_dir}')

## Calculate engineering constants

In [ ]:
# Initialization
results = []
materials = ["EW", "TW", "LW", "GR"]

for mat in materials:
    mat_dir = base_dir / mat
    csv_file = mat_dir / f"{mat}_elastic_compliance_coeffs.csv"
    if not csv_file.is_file():
        print(f"⚠️  Missing CSV for {mat}: {csv_file.name}")
        continue

    # read one-row CSV into a Series
    s = pd.read_csv(csv_file).iloc[0]

    # map D→S
    S11, S22, S33 = s["D11"], s["D22"], s["D33"]
    S44, S55, S66 = s["D44"], s["D55"], s["D66"]
    S12, S13, S23 = s["D12"], s["D13"], s["D23"]

    # compute engineering constants
    E1, E2, E3 = 1/S11, 1/S22, 1/S33
    G23, G13, G12 = 1/S44, 1/S55, 1/S66
    nu12 = -S12 / S11
    nu13 = -S13 / S11
    nu23 = -S23 / S22

    results.append({
        "material": mat,
        "E1": E1, "E2": E2, "E3": E3,
        "G23": G23, "G13": G13, "G12": G12,
        "nu12": nu12, "nu13": nu13, "nu23": nu23
    })
df = pd.DataFrame(
    results,
    columns=["material","E1","E2","E3","G23","G13","G12","nu12","nu13","nu23"]
)
# round all numeric columns to 3 decimal places
df.loc[:, df.columns != "material"] = df.loc[:, df.columns != "material"].round(6)

# inspect
display(df)

## Collect and save data

In [ ]:
out_path = os.path.join(base_dir, "collect_engineering_constants.csv")
df.to_csv(out_path, index=False)
print(f"\n✅ Saved all engineering constants to: {out_path}")